In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-movie-reviews/IMDB Dataset.csv


In [20]:
#This is for creating a sample Dataframe from the IMDB Movie Reviews.

file_path = "/kaggle/input/imdb-movie-reviews/IMDB Dataset.csv"

reviews_df = pd.read_csv(file_path)

#This prints out the first 5 reviews and the last 5 reviews just for a quick review of the data.

print ("First five reviews:")
print(reviews_df.head())
print()
print ("Last five reviews:")
print(reviews_df.tail())

First five reviews:
                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive

Last five reviews:
                                                  review sentiment
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative


In [21]:
#This is to print out a summary of the dataframe to inspect if there are any missing values.

print(reviews_df.info())

print(reviews_df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB
None
                                                   review sentiment
count                                               50000     50000
unique                                              49582         2
top     Loved today's show!!! It was a variety and not...  positive
freq                                                    5     25000


In [22]:
#Check the number of nulls in the data

print (reviews_df.isnull().sum())

review       0
sentiment    0
dtype: int64


In [23]:
#To make sure the words are treated the same no matter how they are displayed as.

reviews_df['review'] = reviews_df['review'].str.lower()

In [24]:
#Same as the previous code block.

import re

reviews_df['review'] = reviews_df['review'].apply(lambda x: re.sub(r'[^a-z\s]', '', x))

In [25]:
#Error happens because of server issues, this code still works after I applied fix

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# This was a fix for the network error on Kaggle
download_dir = "/kaggle/working/"

nltk.download('stopwords', download_dir=download_dir)
nltk.download('punkt', download_dir=download_dir)

nltk.data.path.append(download_dir)

# Getting the list of stop worrds in English
stop_words = set(stopwords.words('english'))

def process_text(text):
    # Tokenize the text into individual words
    tokens = word_tokenize(text)
    # Remove stop words and join the remaining words back into a string
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    return " ".join(filtered_tokens)

reviews_df['cleaned_review'] = reviews_df['review'].apply(process_text)

print(reviews_df.head())

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
[nltk_data] Error loading punkt: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


                                              review sentiment  \
0  one of the other reviewers has mentioned that ...  positive   
1  a wonderful little production br br the filmin...  positive   
2  i thought this was a wonderful way to spend ti...  positive   
3  basically theres a family where a little boy j...  negative   
4  petter matteis love in the time of money is a ...  positive   

                                      cleaned_review  
0  one reviewers mentioned watching oz episode yo...  
1  wonderful little production br br filming tech...  
2  thought wonderful way spend time hot summer we...  
3  basically theres family little boy jake thinks...  
4  petter matteis love time money visually stunni...  


In [26]:
#Vectorrization so the Machine can understand it better.
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=5000)

X = tfidf_vectorizer.fit_transform(reviews_df['cleaned_review'])

In [27]:
#Model building and training
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

y = reviews_df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [28]:
#Model evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8869

Classification Report:
               precision    recall  f1-score   support

    negative       0.90      0.87      0.88      4961
    positive       0.88      0.90      0.89      5039

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000

